<a href="https://colab.research.google.com/github/SterlingHayden/Grad-CAM/blob/main/Large-Scale-Models-In-Pyomo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 6: Formulating Large-Scale Models in Pyomo

## Objectives

Students will be skilled at
1.   implementing and solving large-scale (indexed) models in Pyomo, and outputing/interpreting their results

## References

1. [Pyomo Documentation](https://pyomo.readthedocs.io/en/stable/)

## Example 1: Indexed Resource-Allocation Model

### Linear Programming Formulation

**Sets**

* $\{1,2,\ldots,m\}$ = items that can be produced (indexed by $i$)
* $\{1,2,\ldots,n\}$ = resources required to produce each item (indexed by $j$)

**Parameters**

* $p_i = $ profit per unit of item $i$ produced ($i = 1,2,\ldots,m$)
* $a_{ij} = $ units of resource $j$ required to produce one unit of item $i$ ($i = 1,2,\ldots,m; \ j = 1,2,\ldots,n$)
* $u_j =$  units of resource $j$ available ($j = 1,2,\ldots,n$)

**Decision Variables**

* $x_i = $ number of units of item $i$ to produce ($i = 1,2,\ldots,m$)

**Model**

$\begin{align}
\max \  & \sum_{i=1}^m p_ix_i & & \textrm{(total profit)}\\
\textrm{s.t.}\ & \sum_{i = 1}^m a_{ij} x_i \leq u_j, \ \forall j = 1,2,\ldots,n && \textrm{(limited amount of resource $j$)}\\
& x_i \geq 0, \ \forall i = 1,2,\ldots,m &&
\end{align}$



### Example Data

In [ ]:
!pip install -q pyomo

In [ ]:
!apt-get install -y -qq coinor-cbc

In [ ]:
I = [1,2,3,4,5]  # list defining a set of items (indexed by i)
J = [1,2]        # list defining a set of resources (indexed by j)

p = {1: 10,      # dictionary that contains the profit per unit of item i
     2: 15,
     3: 35,
     4: 25,
     5: 50}

u = {1: 100,     # dictionary that contains the number of units of resource j available
     2: 300}

a = {(1,1): 0.5, # dictionary that contains at (i,j) the number of units of resource j required to produce a unit of item i
     (1,2): 0.7,
     (2,1): 1.1,
     (2,2): 1.5,
     (3,1): 2.1,
     (3,2): 3.7,
     (4,1): 0.6,
     (4,2): 2.7,
     (5,1): 2.0,
     (5,2): 5.1}

In [ ]:
# don't use too much resource 2
# 0.7*x[1] + 1.5*x[2] + 3.7*x[3] + 2.7*x[4] + 5.1*x[5] <= 300
a[(1,1)]

### Building a Concrete Pyomo Model with Single-Indexed Variables and Sums

In [ ]:
#Import Pyomo and define a concrete model
import pyomo.environ as pyo
model = pyo.ConcreteModel()

In [ ]:
# Define index sets for items and resources
model.I = pyo.Set(initialize = I)
model.J = pyo.Set(initialize = J)
model.I.pprint()
model.J.pprint()

In [ ]:
# Define parameters p, a, and u
model.p = pyo.Param(model.I, initialize=p)
model.u = pyo.Param(model.J, initialize=u)
model.a = pyo.Param(model.I, model.J, initialize=a)

In [ ]:
#Add x-variables to model
model.x = pyo.Var(model.I, domain=pyo.NonNegativeReals)

In [ ]:
#Define the objective
def obj_rule_ex1(model):
  return sum(model.p[i]*model.x[i] for i in model.I)

model.obj = pyo.Objective(rule = obj_rule_ex1, sense=pyo.maximize)

In [ ]:
model.obj.pprint()

In [ ]:
#Define the constraints
def resource_rule(model, j):
  return sum(model.a[i,j]*model.x[i] for i in model.I) <= model.u[j]

model.resource_limit = pyo.Constraint(model.J, rule=resource_rule)

In [ ]:
model.pprint()

### Solving the Model

In [ ]:
#Declare the solver as CBC
opt = pyo.SolverFactory('cbc')

#Solve the model
opt.solve(model).write()

### Retrieving the Results

In [ ]:
#Access the variable values using pprint
model.x.pprint()

## Example 2: Transportation Problem

This example demonstrates formulating and solving a Pyomo model in which variables and constraints have two indices.

### Problem Description (Adapted from Winston, 2004)

A company supplies goods to customers, who each require 30 units.  The company has four warehouses.  Warehouse 1 has 70 units available and the remaining warehouses each have 50 units available.  The per-unit cost of shipping goods from warehouse to customer are provided in the following table.

\begin{array}{ccccccc} \hline
 & \mbox{Cust. 1} & \mbox{Cust. 2} & \mbox{Cust. 3} & \mbox{Cust. 4}& \mbox{Cust. 5} & \mbox{Cust. 6} \\ \hline
\mbox{Warehouse 1} & 15 & 35 & 25 & 20 & 30 & 40\\
\mbox{Warehouse 2} & 10 & 50 & 35 & 20 & 25 & 45\\
\mbox{Warehouse 3} & 20 & 55 & 40 & 20 & 25 & 35\\
\mbox{Warehouse 4} & 25 & 40 & 30 & 35 & 20 & 25\\ \hline
\end{array}

How many units should the company ship from each warehouse to each customer to minimize cost?








In [ ]:
I = [1, 2, 3, 4] # set of warehouses, indexed by i
J = [1, 2, 3, 4, 5, 6] # set of warehouses, indexed by j

c = {(1,1): 15, # dictionary containing the per-unit shipping costs from table
     (1,2): 35,
     (1,3): 25,
     (1,4): 20,
     (1,5): 30,
     (1,6): 40,
     (2,1): 10,
     (2,2): 50,
     (2,3): 35,
     (2,4): 20,
     (2,5): 25,
     (2,6): 45,
     (3,1): 20,
     (3,2): 55,
     (3,3): 40,
     (3,4): 20,
     (3,5): 25,
     (3,6): 35,
     (4,1): 25,
     (4,2): 40,
     (4,3): 30,
     (4,4): 35,
     (4,5): 20,
     (4,6): 25}

b = {1: 70, # dictionary containing the supply of each warehouse
     2: 50,
     3: 50,
     4: 50}

d = {1: 30, # dictionary containing the demand of each customer
     2: 30,
     3: 30,
     4: 30,
     5: 30,
     6: 30}

### Linear Programming Formulation

**Sets**

* $\{1,2,\ldots,m\}$ = set of warehouses (indexed by $i$)
* $\{1,2,\ldots,n\}$ = set of customers (indexed by $j$)

**Parameters**

* $b_i =$  supply at warehouse $i$ ($i = 1,2,\ldots,m$)
* $d_j =$  demand of customer $j$ ($j = 1,2,\ldots,n$)
* $c_{ij} = $ cost per unit shipped from warehouse $i$ to customer $j$ ($i = 1,2,\ldots,m; \ j = 1,2,\ldots,n$)

**Decision Variables**

* $x_{ij} = $ number of units shipped from warehouse $i$ to customer $j$ ($i = 1,2,\ldots,m; \ j = 1,2,\ldots,n$)

**Model**

$\begin{align}
\min \  & \sum_{i=1}^m \sum_{j = 1}^n c_{ij}x_{ij} & & \textrm{(total shipping cost)}\\
\textrm{s.t.}\ & \sum_{j = 1}^n x_{ij} \leq b_i, \ \forall i = 1,2,\ldots,m && \textrm{(supply limit at warehouse $i$)}\\
&  \sum_{i = 1}^m x_{ij} \geq d_j, \ \forall j = 1,2,.\ldots,n && \textrm{(demand requirement for cust. $j$)} \\
& x_{ij} \geq 0, \ \forall i = 1,2,\ldots,m, \ \forall j = 1,2,\ldots,n &&
\end{align}$



### Building a Concrete Pyomo Model with Double-Indexed Variables and Sums

In [ ]:
#Clear the model
model.clear()
model.pprint()

In [ ]:
#Define index sets for warehouses and customers
model.I = pyo.Set(initialize = I)
model.J = pyo.Set(initialize = J)
model.I.pprint()
model.J.pprint()

In [ ]:
#Define parameters c, b, and d
model.c = pyo.Param(model.I, model.J, initialize=c)
model.b = pyo.Param(model.I, initialize=b)
model.d = pyo.Param(model.J, initialize=d)

In [ ]:
#Add x-variables to model
model.x = pyo.Var(model.I, model.J, domain=pyo.NonNegativeReals)

In [ ]:
#Define the objective
def obj_rule_ex2(model):
  return sum(model.c[i,j]*model.x[i,j] for i in model.I for j in model.J)

model.obj = pyo.Objective(rule=obj_rule_ex2, sense=pyo.minimize)

In [ ]:
model.obj.pprint()

In [ ]:
#Add the supply limit constraints
def supply_limit_rule(model,i):
  return sum(model.x[i,j] for j in model.J) <= model.b[i]

model.supply_limit = pyo.Constraint(model.I, rule=supply_limit_rule)

In [ ]:
model.supply_limit.pprint()

In [ ]:
#Add the demand requirement constraints
def demand_rqmt_rule(model,j):
  return sum(model.x[i,j] for i in model.I) >= model.d[j]

model.deman_rqmt = pyo.Constraint(model.J, rule=demand_rqmt_rule)

In [ ]:
model.deman_rqmt.pprint()

### Solving the Model

In [ ]:
#Declare the solver as CBC
opt = pyo.SolverFactory('cbc')

#Solve the model
opt.solve(model).write()

### Retrieving the Results

In [ ]:
#Access the variable values using pprint
model.x.pprint()

In [ ]:
#Display a customized table of output
print('%4s %4s %5s %5s' % ('from','to','cost','units'))
for i in model.I:
  for j in model.J:
    print('%4s %4s %5.2f %5.2f' % (i,j,model.c[i,j],pyo.value(model.x[i,j])))

In [ ]:
#Display a customized table of output, but omit rows where the x-value is zero
print('%4s %4s %5s %5s' % ('from','to','cost','units'))
for i in model.I:
  for j in model.J:
    if (pyo.value(model.x[i,j]) > 0):
      print('%4s %4s %5.2f %5.2f' % (i,j,model.c[i,j],pyo.value(model.x[i,j])))

## Revisiting Example 2: Reading the Data from CSV

### Reading the Data using Pandas

In [ ]:
# upload the L6-transportation_data csv files (one for transp costs, one for supplies, one for demands)
from google.colab import files
files.upload()

In [ ]:
#Read cost data from CSV
import pandas as pd
# (path to data file, excel sheet in data file, row index of headers, col index of row labels)
cost_df = pd.read_csv('L6-transportation_data_transp_costs.csv', header = 0, index_col = 0)
cost_df.head()

In [ ]:
#Read supplies data from CSV
supplies_df = pd.read_csv('L6-transportation_data_supplies.csv', header = 0, index_col = 0)
supplies_df.head()

In [ ]:
#Read demands data from CSV
demands_df = pd.read_csv('L6-transportation_data_demands.csv', header = 0, index_col = 0)
demands_df.head()

### Building and Solving the Model

In [ ]:
#Clear the model
model.clear()
model.pprint()

In [ ]:
print(list(cost_df.index))
print(list(cost_df.columns))

In [ ]:
#Redefine the lists and dictionaries, this time using the data read in from CSV
I = list(cost_df.index)
J = list(cost_df.columns)
c = {(i,j): cost_df.at[i,j] for i in I for j in J}
b = {i: supplies_df.at[i, 'Supply'] for i in I}
d = {j: demands_df.at[j, 'Demand'] for j in J}

In [ ]:
#Define index sets for warehouses and customers
model.I = pyo.Set(initialize = I) # set of warehouses
model.J = pyo.Set(initialize = J) # set of customers

In [ ]:
#Define parameters c, b, and d
model.b = pyo.Param(model.I,initialize = b)
model.d = pyo.Param(model.J,initialize = d)
model.c = pyo.Param(model.I,model.J,initialize = c)

In [ ]:
#Add x-variables to model
model.x = pyo.Var(model.I, model.J, domain=pyo.NonNegativeReals)

In [ ]:
#Define the objective
def obj_rule_ex2(model):
  return sum(model.c[i,j]*model.x[i,j] for i in model.I for j in model.J)

model.obj = pyo.Objective(rule=obj_rule_ex2, sense=pyo.minimize)

In [ ]:
#Add the supply limit constraints
def supply_limit_rule(model,i):
  return sum(model.x[i,j] for j in model.J) <= model.b[i]

model.supply_limit = pyo.Constraint(model.I, rule=supply_limit_rule)

In [ ]:
#Add the demand requirement constraints
def demand_rqmt_rule(model,j):
  return sum(model.x[i,j] for i in model.I) >= model.d[j]

model.deman_rqmt = pyo.Constraint(model.J, rule=demand_rqmt_rule)

In [ ]:
#Declare the solver as CBC
opt = pyo.SolverFactory('cbc')

#Solve the model
opt.solve(model).write()

### Retrieving the Results

In [ ]:
#Display a customized table of output
print('%12s %12s %5s %5s' % ('from','to','cost','units'))
for i in model.I:
  for j in model.J:
    print('%12s %12s %5.2f %5.2f' % (i,j,model.c[i,j],pyo.value(model.x[i,j])))

## Revisting Example 2: Re-solving the model after changing a parameter's value.

Currently c[Wearhouse 4, Customer 5] = 20. What if this value increases.

In [ ]:
model.clear()

In [ ]:
#Define index sets for warehouses and customers
model.I = pyo.Set(initialize = I) # set of warehouses
model.J = pyo.Set(initialize = J) # set of customers

#Define parameters c, b, and d
model.b = pyo.Param(model.I,initialize = b)
model.d = pyo.Param(model.J,initialize = d)
model.c = pyo.Param(model.I,model.J,initialize = c, mutable=True) #switching to mutable

#Add x-variables to model
model.x = pyo.Var(model.I, model.J, domain=pyo.NonNegativeReals)

#Define the objective
def obj_rule_ex2(model):
  return sum(model.c[i,j]*model.x[i,j] for i in model.I for j in model.J)
model.obj = pyo.Objective(rule=obj_rule_ex2, sense=pyo.minimize)

#Add the supply limit constraints
def supply_limit_rule(model,i):
  return sum(model.x[i,j] for j in model.J) <= model.b[i]
model.supply_limit = pyo.Constraint(model.I, rule=supply_limit_rule)

#Add the demand requirement constraints
def demand_rqmt_rule(model,j):
  return sum(model.x[i,j] for i in model.I) >= model.d[j]
model.deman_rqmt = pyo.Constraint(model.J, rule=demand_rqmt_rule)

In [ ]:
#Declare the solver as CBC
opt = pyo.SolverFactory('cbc')

#Solve the model
opt.solve(model).write()

In [ ]:
# solve for each value of c[4,5] in [20,...,27]
for val in range(20,28):
  model.c[('Warehouse 4', 'Customer 5')] = val
  opt.solve(model)
  print("c[4,5] = ", pyo.value(model.c[('Warehouse 4', 'Customer 5')]), ", objective = ", pyo.value(model.obj))